### 카메라에서 실시간으로 영상을 캡처

In [2]:
import cv2  

# 웹캠 연결 및 비디오 캡처 객체 생성 (0은 기본 카메라를 나타냄)
cam = cv2.VideoCapture(0)

# 웹캠이 열려있는 동안 계속 반복
while True:
    # 첫 번째 반환 값은 프레임이 정상적으로 읽혔는지 여부
    # 두 번째 반환 값은 읽어온 프레임
    check, frame = cam.read()

    # 읽어온 프레임을 화면에 출력
    cv2.imshow("webcam", frame)

    # 키보드 입력을 확인합니다. 입력이 있으면 루프를 종료
    key = cv2.waitKey(1)
    if key != -1:
        break

# 웹캠 연결을 종료하고, 모든 OpenCV 창을 닫기
cam.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## YOLO v5

### 1. 필요한 라이브러리 불러오기

In [3]:
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
"""
Run inference on images, videos, directories, streams, etc.

Usage - sources:
    $ python path/to/detect.py --weights yolov5s.pt --source 0              # webcam
                                                             img.jpg        # image
                                                             vid.mp4        # video
                                                             path/          # directory
                                                             path/*.jpg     # glob
                                                             'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                                                             'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream

Usage - formats:
    $ python path/to/detect.py --weights yolov5s.pt                 # PyTorch
                                         yolov5s.torchscript        # TorchScript
                                         yolov5s.onnx               # ONNX Runtime or OpenCV DNN with --dnn
                                         yolov5s.xml                # OpenVINO
                                         yolov5s.engine             # TensorRT
                                         yolov5s.mlmodel            # CoreML (macOS-only)
                                         yolov5s_saved_model        # TensorFlow SavedModel
                                         yolov5s.pb                 # TensorFlow GraphDef
                                         yolov5s.tflite             # TensorFlow Lite
                                         yolov5s_edgetpu.tflite     # TensorFlow Edge TPU
"""

import argparse
import os
import sys
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

from utils.augmentations import letterbox
import numpy as np

C:\Users\sangwoo\anaconda3\envs\pytorch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 객체 검출 및 분류를 수행하기 전에 필요한 여러 설정값들을 초기화

In [4]:
import cv2

imgsz = (640, 640)  # 이미지 크기
weights = "./runs/train/exp8/weights/best.pt"  # 학습된 모델의 가중치 파일 경로
# weights = "./yolov5m.pt"
img_path = "./data_iron/images/test"  # 테스트 이미지 경로
device = ''  # 디바이스 설정 (빈 문자열은 자동으로 CPU 또는 GPU를 선택)
dnn = False  # DNN 사용 여부
data = False  # 데이터 사용 여부
half = False  # Half-precision 사용 여부
visualize = False  # 시각화 사용 여부
view_img = True  # 이미지 보기 설정
augment = False  # 이미지 증강 사용 여부
conf_thres = 0.90  # 검출에 필요한 최소 신뢰도 (confidence) 임계값
iou_thres = 0.45  # IOU (Intersection Over Union) 임계값
classes = [i for i in range(6)]  # 분류할 클래스 인덱스
agnostic_nms = False  # 클래스에 상관없이 NMS (Non-Maximum Suppression) 적용 여부
max_det = 1000  # 최대 검출 개수
webcam = False  # 웹캠 사용 여부
project = 'runs/train'  # 프로젝트 경로
name = 'eee'  # 이름 설정
exist_ok = False  # 기존 파일이 존재할 경우 덮어쓰기 허용 여부
save_txt = False  # 결과를 텍스트 파일로 저장 여부
save_crop = False  # 검출된 객체를 잘라낸 이미지로 저장 여부
line_thickness = 20  # 경계선 두께
nosave = False  # 결과를 저장하지 않는 여부
hide_labels = False  # 레이블 숨기기 여부
hide_conf = False  # 신뢰도 숨기기 여부

## 이미지로 예측하기(일반카메라)

In [5]:
# 이미지로 결과 보기 설정
source = str(img_path)
save_img = not nosave and not source.endswith('.txt')  # save inference images
print(save_img)

# 결과를 저장할 디렉토리 생성
save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
(save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

# 모델 불러오기
device = select_device(device)
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)  # weights에 경로 넣기
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # check image size

# 이미지 데이터셋 불러오기
dataset = LoadImages(str(img_path), img_size=imgsz, stride=stride, auto=pt)  # source = image

# 배치 크기 설정
bs = 1  # batch_size
vid_path, vid_writer = [None] * bs, [None] * bs

# 모델 워밍업 (첫 번째 추론에 걸리는 시간을 줄이기 위해)
model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
dt, seen = [0.0, 0.0, 0.0], 0

#
picture_num = 0

YOLOv5  v7.0-153-gff6a9ac Python-3.8.16 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)



True


Fusing layers... 
Model summary: 212 layers, 20889303 parameters, 0 gradients


#### test 이미지 데이터 경로의 test 이미지 예측

In [6]:
for path, im, im0s, vid_cap, s in dataset:
    picture_num += 1
    
    t1 = time_sync()
    im = torch.from_numpy(im).to(device) # 이미지를 PyTorch 텐서로 변환하고, GPU로 전송
    im = im.half() if model.fp16 else im.float()   # FP16 사용 여부에 따라 이미지 데이터 타입 변환
    im /= 255  # 이미지 정규화
    if len(im.shape) == 3:
        im = im[None]  # batch 차원 추가
    t2 = time_sync()
    dt[0] += t2 - t1

    # Inference
    visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False  # 시각화 여부에 따라 시각화 결과 저장 경로 설정
    pred = model(im, augment=augment, visualize=visualize) # 추론 결과 예측
    t3 = time_sync()
    dt[1] += t3 - t2

    # NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)  # Non-Maximum Suppression 적용
    dt[2] += time_sync() - t3 
    print(pred[0])

    # Process predictions
    for i, det in enumerate(pred):  # per image
        seen += 1
        # 웹캠 모드인 경우
        if webcam:  # batch_size >= 1
            p, im0, frame = path[i], im0s[i].copy(), dataset.count
            s += f'{i}: '
        # 일반 이미지/비디오 모드인 경우
        else:
            p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

       # 경로를 Path 객체로 변환
        p = Path(p)
        # 이미지 저장 경로
        save_path = str(save_dir / p.name)  # im.jpg
        # 라벨 저장 경로
        txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
        s += '%gx%g ' % im.shape[2:]  # 출력할 문자열
        # normalization gain whwh
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]
        imc = im0.copy() if save_crop else im0  # for save_crop
        annotator = Annotator(im0, line_width=line_thickness, example=str(names))

        # 예측 결과가 있는 경우
        if len(det):
            # bbox를 img_size에서 im0 크기로 조정
            det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

            # 결과 출력
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # 클래스당 감지 개수
                s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # 문자열에 추가

            # 결과 저장
            for *xyxy, conf, cls in reversed(det):
                if save_txt:  # 파일에 쓰기
                    # normalized xywh
                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()
                    line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # 라벨 형식
                    with open(f'{txt_path}.txt', 'a') as f:
                        f.write(('%g ' * len(line)).rstrip() % line + '\n')

                if save_img or save_crop or view_img:  # bbox를 이미지에 추가
                    c = int(cls)  # 정수형 클래스
                    label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                    annotator.box_label(xyxy, label, color=colors(c, True))
                if save_crop:
                    save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

        # 결과 스트리밍
        im0 = annotator.result()
        if view_img:
            cv2.imshow(str(p), im0)
            key = cv2.waitKey(1)

            if key != -1:
                break;

        # Save results (image with detections)
        if save_img:
            if dataset.mode == 'image': # 이미지 모드인 경우
                cv2.imwrite(save_path, im0)
            else:  # '비디오 또는 스트림 모드인 경우
                if vid_path[i] != save_path:  # 새로운 비디오인 경우
                    vid_path[i] = save_path
                    if isinstance(vid_writer[i], cv2.VideoWriter):
                        vid_writer[i].release()  # release previous video writer
                    if vid_cap:  # video
                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    else:  # stream
                        fps, w, h = 30, im0.shape[1], im0.shape[0]
                    save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                    vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                vid_writer[i].write(im0)

    # Print time (inference-only)
    LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')
    
    if picture_num == 1:
        break

image 1/46 C:\Users\sangwoo\Lecture\202304_CVDL\git_code\Day2\yolov5\data_iron\images\test\IMG_1920.jpg: 480x640 3 class_As, Done. (0.014s)


tensor([[390.95316, 236.05774, 489.17355, 326.57263,   0.94924,   0.00000],
        [332.48697, 205.76053, 373.33597, 242.72012,   0.94067,   0.00000],
        [320.96039, 302.56650, 387.36670, 362.36887,   0.93916,   0.00000]], device='cuda:0')


## 실시간 예측하기(일반카메라)

In [ ]:
## 이건 영상으로 보기

import cv2

imgsz = (640, 640)
weights = "./runs/train/exp8/weights/best.pt"
device=''
visualize=False
augment=False
# conf_thres=0.25  # 0.25
# iou_thres=0.45 # 0.45

conf_thres = 0.90  # 검출에 필요한 최소 신뢰도 (confidence) 임계값
iou_thres = 0.45  # IOU (Intersection Over Union) 임계값

dnn = False
data = False
half = False
classes = [i for i in range(6)]  # 분류할 클래스 인덱스
agnostic_nms = False
max_det=1000
webcam = False#img_path.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
view_img=True # False # 
# project= 'runs/train'
# name='eee'
# exist_ok=False
# save_txt=False
save_crop=False
line_thickness=5
save_txt=False
# nosave=False
hide_labels=False
hide_conf=False

In [14]:
# 모델 불러오기
device = select_device(device)
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half) # weights에 경로 넣기
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # 이미지 크기 확인 및 업데이트

# 웹캠 초기화
cap = cv2.VideoCapture(0)

while True:
    # 웹캠에서 프레임 캡처
    ret_val, img0 = cap.read()

    bs = 1  # 배치 크기
    vid_path, vid_writer = [None] * bs, [None] * bs

    model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # 워밍업
    dt, seen = [0.0, 0.0, 0.0], 0

    # 패딩된 리사이즈
    img = letterbox(img0, imgsz, stride=stride, auto=pt)[0]

    # 변환
    img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img = np.ascontiguousarray(img)

    im = img.copy()
    im0s = img0.copy()

    t1 = time_sync()
    im = torch.from_numpy(im).to(device)
    im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # 배치 차원 확장
    t2 = time_sync()
    dt[0] += t2 - t1

    # 추론
    visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
    pred = model(im, augment=augment, visualize=visualize)
    t3 = time_sync()
    dt[1] += t3 - t2

    # NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
    dt[2] += time_sync() - t3

    # 예측 결과 처리
    for i, det in enumerate(pred):  # 이미지 당
        seen += 1
        if webcam:  # 배치 크기 >= 1
            p, im0, frame = None, im0s[i].copy(), None
            s += f'{i}: '
        else:
            p, im0, frame = None, im0s.copy(), None
        
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # 정규화 획득 whwh
        imc = im0.copy() if save_crop else im0  # save_crop 용
        annotator = Annotator(im0, line_width=line_thickness, example=str(names))
        if len(det):
            # 박스 좌표를 img_size에서 im0 크기로 변환
            det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

            # 결과 출력
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # 클래스별 탐지 수
                # s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # 문자열 추가

            # 결과 저장
            for *xyxy, conf, cls in reversed(det):
                if save_txt:  # 파일에 저장
                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # 정규화된 xywh
                    line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # 라벨 형식
                    with open(f'{txt_path}.txt', 'a') as f:
                        f.write(('%g ' * len(line)).rstrip() % line + '\n')

                if save_img or save_crop or view_img:  # 이미지에 bbox 추가
                    c = int(cls)  # 정수 클래스
                    label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                    annotator.box_label(xyxy, label, color=colors(c, True))
                if save_crop:
                    save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

        # 결과 스트리밍
        im0 = annotator.result()
        if view_img:
            cv2.imshow(str(p), im0)
            key = cv2.waitKey(1)

            if key != -1:
                break

cv2.destroyAllWindows()
cv2.waitKey(1)

YOLOv5  v7.0-153-gff6a9ac Python-3.8.16 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)

Fusing layers... 
Model summary: 212 layers, 20889303 parameters, 0 gradients


KeyboardInterrupt: 